<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/crawler/Finance_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html
import re

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    page = str(page)
    url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + page
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    div = bs_obj.find_all('div', {'class':'hotNewsList'})
    return [html.unescape(x['href']) for x in div[0].find_all('a')]

  def get_main(self, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/mainnews.nhn'
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return [x.find_all('a')[0]['href'] for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    
  def get_breaking(self):
    pass

  # 시황 전망: 401
  # 기업 종목분석: 402
  # 해외 증시: 403
  # 채권 선물: 404
  # 공시 메모: 406
  # 환율: 429
  def get_focus(self, code=403, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3='\
     + str(code) +  '&date=' + str(date) + '&page=' + str(page)
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    temp = [html.unescape(x.find_all('a')[0]['href']) for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    # 업데이트 가능
    return [x.replace('§', '&sect') for x in temp]

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    return title, subheading, text

  # 특징주
  def get_search(self, keyword='%C6%AF%C2%A1%C1%D6', start_date='1', end_date='1'):
    start_date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if start_date == '1' else start_date
    end_date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if end_date == '1' else end_date

    li = []
    for page in range(1,1000):
      url = 'https://finance.naver.com/news/news_search.nhn?rcdate=&q=' + keyword +\
      '&sm=title.basic&pd=4&stDateStart=' + start_date + '&stDateEnd=' + end_date + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")
      hrefs = [x.find_all('a')[0]['href'] for x in bs_obj.find_all(re.compile('d*'), {'class':"articleSubject"})] 
      if not hrefs:
        break
      li += hrefs

    return li

In [2]:
from multiprocessing import Pool

news = News()
news_list = news.get_search('%C6%AF%C2%A1%C1%D6', '2021-06-11', '2021-06-11')
with Pool(50) as p:
    li = p.map(news.get_article, news_list)
li = np.array(li)

dic = {'Title':li[:,0], 'Subheading':li[:,1], 'Text':li[:,2]}
df = pd.DataFrame.from_dict(dic)
df

,Title,Subheading,Text
0,"[특징주] 바이오톡스텍, 정부 연매출 1조 블록버스터 신약 개발…임상인프라 지원 수...",▶ 날로먹고 구워먹는 금융이슈 [파인애플]▶ 모(毛)아 모아 [모아시스]▶ 헉! 소...,[파이낸셜뉴스] 정부가 국내 제약사들을 대상으로 오는 2030년까지 연매출 1조원 ...
1,[특징주]이스타항공 인수전 참여 하림...3일째 급등 '52주 신고가',,[아시아경제 박지환 기자] 이스타항공 인수전에 참여한 하림 주가가 3거래일 연속 급...
2,"[특징주] 솔루스첨단소재2우B, 10%대 급등",,[헤럴드경제=이세진 기자] 솔루스첨단소재2우B 주가가 11일 장 초반 10%대 상승...
3,"[특징주]엘티씨, 3000조 수소시장 K수소 동맹 구축에 국내 유일 SOFC기술 부각",▶ 날로먹고 구워먹는 금융이슈 [파인애플]▶ 모(毛)아 모아 [모아시스]▶ 헉! 소...,"[파이낸셜뉴스]현대자동차그룹과 SK그룹, 포스코그룹, 효성그룹 등 국내 수소 관련 ..."
4,"[특징주]나노스, 쌍방울 이스타항공 인수 기대감에 '강세'",,[아시아경제 장효원 기자] 쌍방울그룹 계열사인 나노스가 쌍방울그룹의 이스타항공 인수...
5,"[특징주] 뉴인텍, 41兆 투자 'K-수소동맹' 참여 현대차에 부품 공급 소식 부각",,"[아시아경제 이민우 기자] 뉴인텍주가가 강세다. 현대차그룹, SK그룹, 포스코그룹,..."
6,[특징주] 수소전기차 부품株 수소기업협의체 추진 소식에 고공행진,"유니크, 에스퓨얼셀 등 10%대 급등","유니크, 에스퓨얼셀 등 10%대 급등현대차그룹이 SK그룹, 포스코그룹, 효성그룹과 ..."
7,"[특징주]동아화성, 수소 시대 비상 준비끝…수소에 꽂힌 바이든",,[아시아경제 박형수 기자] 동아화성이 강세다. 한국 수소산업 생태계를 이끌어온 현대...
8,"[특징주]신풍제약, 美 코로나 경구치료제 계약에 이틀째 급등세",,[사진=신풍제약][헤럴드경제=김현경 기자] 미국이 코로나바이러스감염증-19(코로나1...
9,"[특징주] 현대이지웰, 현대백화점그룹 시너지 전망에 강세",,[한국경제TV 이민재 기자]현대이지웰이 현대백화점 그룹 계열사로 편입돼 향후 시너지...


In [3]:
'''dic = {'Title':[], 'Subheading':[], 'Text':[]}

for i in news.get_search('%C6%AF%C2%A1%C1%D6', '2021-06-11', '2021-06-11'):
  title, subheading, text = news.get_article(i)
  dic['Title'].append(title)
  dic['Subheading'].append(subheading)
  dic['Text'].append(text)
  
pd.DataFrame.from_dict(dic)'''

"dic = {'Title':[], 'Subheading':[], 'Text':[]}\n\nfor i in news.get_search('%C6%AF%C2%A1%C1%D6', '2021-06-11', '2021-06-11'):\n  title, subheading, text = news.get_article(i)\n  dic['Title'].append(title)\n  dic['Subheading'].append(subheading)\n  dic['Text'].append(text)\n  \npd.DataFrame.from_dict(dic)"